In [231]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import os

# Analysis of Simulation Benchmarking Experiments

The benchmarking experiments were run on  a Dell XPS-15. The metrics reported in the paper come from these experiments.

In [232]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)
            

In [233]:
def replace_e2e(row):
    if(row=="e2e_image"):
        return "VBN"
    elif(row=="e2e"):
        return "LBC"
    else:
        return row.upper()
def replace_obs(row):
    if(row==0):
        return "No" 
    else: 
        return "Yes"

In [234]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000
all_dfs = all_dfs.reset_index().drop(columns='index')
all_dfs

,time_string,moet,mean_et,avg_iterations,obstacle_presence,ml_usage,sc_usage,ml_controller,speed,runtime
0,01/04/2021/17:06:07,8.478,6.64321,3.96654,No,94.933493,5.066507,VBN,0.5,10
1,01/04/2021/17:07:36,8.315,6.60067,3.97587,No,88.899489,11.100511,VBN,0.5,10
2,01/04/2021/17:09:06,8.507,6.43541,3.98251,No,94.946925,5.053075,VBN,0.5,10
3,01/04/2021/17:10:21,8.885,6.63455,3.96435,No,94.508173,5.491827,VBN,0.5,10
4,01/04/2021/17:11:51,8.353,6.52825,3.98548,No,93.997684,6.002316,VBN,0.5,10
...,...,...,...,...,...,...,...,...,...,...
1817,04/04/2021/17:16:40,26.588,14.33750,5.40917,Yes,33.543118,66.456882,ARS,1.5,25
1818,04/04/2021/17:17:54,31.616,13.74030,6.50090,Yes,20.577657,79.422343,ARS,1.5,25
1819,04/04/2021/17:19:23,27.052,14.65300,5.88859,Yes,27.717371,72.282629,ARS,1.5,25
1820,04/04/2021/17:20:37,29.132,14.57170,5.66425,Yes,25.339281,74.660719,ARS,1.5,25


### Experiments where the RL controller's error out

In [235]:
all_dfs.iloc[np.where(all_dfs['moet']>(3*all_dfs['mean_et']))].groupby(["ml_controller","speed","runtime"]).count().sort_values(by="avg_iterations",ascending=False)

time_string  moet  mean_et  avg_iterations  \
ml_controller speed runtime                                               
ARS           1.0   10                16    16       16              16   
                    25                14    14       14              14   
              0.5   10                12    12       12              12   
                    25                11    11       11              11   
SAC           1.5   10                 8     8        8               8   
ARS           1.5   10                 5     5        5               5   
LBC           1.5   10                 4     4        4               4   
SAC           0.5   10                 4     4        4               4   
              1.0   10                 4     4        4               4   
LBC           1.0   25                 2     2        2               2   
              1.5   25                 2     2        2               2   
SAC           1.0   25                 2     2        2               2   
              1.5   25                 2     2        2               2   
VBN           1.5   10                 2     2        2               2   
ARS           1.5   25                 1     1        1               1   
LBC           0.5   10                 1     1        1               1   

                             obstacle_presence  ml_usage  sc_usage  
ml_controller speed runtime                                         
ARS           1.0   10                      16        16        16  
                    25                      14        14        14  
              0.5   10                      12        12        12  
                    25                      11        11        11  
SAC           1.5   10                       8         8         8  
ARS           1.5   10                       5         5         5  
LBC           1.5   10                       4         4         4  
SAC           0.5   10                       4         4         4  
              1.0   10                       4         4         4  
LBC           1.0   25                       2         2         2  
              1.5   25                       2         2         2  
SAC           1.0   25                       2         2         2  
              1.5   25                       2         2         2  
VBN           1.5   10                       2         2         2  
ARS           1.5   25                       1         1         1  
LBC           0.5   10                       1         1         1

In [236]:
all_dfs.iloc[np.where(all_dfs['moet']>(3*all_dfs['mean_et']))].groupby(["ml_controller","speed","runtime"]).mean()

moet    mean_et  avg_iterations   ml_usage  \
ml_controller speed runtime                                                     
ARS           0.5   10        44.239250   6.046177        4.776854  68.282428   
                    25        69.442091  13.900918        5.826287  68.068846   
              1.0   10        45.753437   5.925238        5.022412  40.877663   
                    25        76.636500  14.291743        5.834811  47.673014   
              1.5   10        28.910600   6.116434        4.601472  25.457377   
                    25       109.884000  14.950300        5.622950  28.688468   
LBC           0.5   10        25.343000   6.430750        4.059730  88.608321   
              1.0   25        71.665000   8.737285       11.753130  23.365553   
              1.5   10        34.621500   5.702460        6.257467  26.305898   
                    25        41.279500   8.493020       12.195640  23.905999   
SAC           0.5   10        33.950750   6.237705        4.616950  47.771657   
              1.0   10        28.766750   6.302422        4.860673  18.220499   
                    25        55.824500  14.501300        5.310455  19.454566   
              1.5   10        47.038750   6.298569        4.802860  10.545885   
                    25        39.142500  10.837385        9.191220  10.028338   
VBN           1.5   10        14.703500   3.856380       11.103280  23.797070   

                              sc_usage  
ml_controller speed runtime             
ARS           0.5   10       31.717572  
                    25       31.931154  
              1.0   10       59.122337  
                    25       52.326986  
              1.5   10       74.542623  
                    25       71.311532  
LBC           0.5   10       11.391679  
              1.0   25       76.634447  
              1.5   10       73.694102  
                    25       76.094001  
SAC           0.5   10       52.228343  
              1.0   10       81.779501  
                    25       80.545434  
              1.5   10       89.454115  
                    25       89.971662  
VBN           1.5   10       76.202930

In [237]:
all_dfs = all_dfs.drop(all_dfs.iloc[np.where(all_dfs['moet']>(3*all_dfs['mean_et']))].index)

In [238]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence","runtime"]).agg(["mean","std"]),3)
res

moet        mean_et         \
                                                 mean    std    mean    std   
ml_controller speed obstacle_presence runtime                                 
ARS           0.5   No                10       11.004  2.911   5.985  0.257   
                                      25       27.238  1.526  14.075  0.866   
                    Yes               10       10.140  1.999   6.042  0.262   
                                      25       27.566  2.205  14.268  0.773   
              1.0   No                10       14.208  2.390   5.965  0.227   
                                      25       28.380  2.329  14.073  0.512   
                    Yes               10       11.444  2.585   5.920  0.507   
                                      25       28.167  2.870  14.337  0.806   
              1.5   No                10        9.134  1.423   6.273  0.240   
                                      25       27.116  1.718  14.238  0.333   
                    Yes               10       10.463  2.549   6.184  0.267   
                                      25       27.668  1.678  14.327  0.619   
LBC           0.5   No                10        9.027  0.395   6.598  0.092   
                                      25       28.247  0.921  14.498  0.106   
                    Yes               10        9.247  0.497   6.415  0.219   
                                      25       28.719  2.664  14.690  0.212   
              1.0   No                10        9.097  0.494   6.615  0.062   
                                      25       27.852  0.603  14.571  0.201   
                    Yes               10        9.868  1.977   6.350  0.256   
                                      25       28.563  2.132  14.935  0.652   
              1.5   No                10        9.673  1.195   6.675  0.506   
                                      25       27.795  0.755  15.014  1.180   
                    Yes               10        9.837  1.555   6.429  0.225   
                                      25       28.994  2.530  14.919  0.527   
SAC           0.5   No                10        9.816  2.526   6.368  0.186   
                                      25       27.375  1.159  13.986  0.309   
                    Yes               10        8.945  0.723   6.286  0.165   
                                      25       27.660  2.167  14.164  0.469   
              1.0   No                10        9.698  1.746   6.417  0.242   
                                      25       27.390  1.994  13.760  0.502   
                    Yes               10       10.222  1.685   6.144  0.268   
                                      25       27.582  2.431  13.917  1.005   
              1.5   No                10        9.377  1.082   6.294  0.282   
                                      25       27.977  2.111  13.691  0.399   
                    Yes               10       10.090  2.025   6.241  0.320   
                                      25       27.642  1.674  14.030  0.442   
VBN           0.5   No                10        8.497  0.279   6.540  0.176   
                                      25       27.175  0.791  14.444  0.697   
                    Yes               10        8.822  0.703   6.389  0.205   
                                      25       27.496  0.902  14.455  0.286   
              1.0   No                10        8.724  0.377   6.588  0.148   
                                      25       27.392  0.699  14.211  0.221   
                    Yes               10        9.243  1.247   6.309  0.584   
                                      25       27.802  1.884  14.510  0.899   
              1.5   No                10        8.932  0.465   6.554  0.168   
                                      25       27.318  0.937  14.362  0.300   
                    Yes               10        9.219  0.592   6.279  0.562   
                                      25       27.360  0.939  14.603  0.780   

                   

# Generate Latex Tables

In [239]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="Yes"]
#table1 = table1[table1.runtime=='25'].groupby(["ml_controller","runtime"]).mean()
table1 = round(table1.groupby(["ml_controller","runtime"]).mean(),2)
table1
print(table1[table1.columns[2:6]].to_latex())

\begin{tabular}{llrrrr}
\toprule
    &    & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
    &    &    mean &   std &           mean &   std \\
ml\_controller & runtime &         &       &                &       \\
\midrule
ARS & 10 &    6.05 &  0.35 &           4.63 &  0.66 \\
    & 25 &   14.31 &  0.73 &           5.68 &  0.45 \\
LBC & 10 &    6.40 &  0.23 &           4.02 &  0.36 \\
    & 25 &   14.85 &  0.46 &           5.18 &  0.29 \\
SAC & 10 &    6.22 &  0.25 &           4.54 &  0.48 \\
    & 25 &   14.04 &  0.64 &           5.63 &  0.67 \\
VBN & 10 &    6.33 &  0.45 &           4.30 &  1.13 \\
    & 25 &   14.52 &  0.66 &           5.27 &  0.46 \\
\bottomrule
\end{tabular}



/home/musaup/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4152: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [240]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="No"]
table1 = round(table1.groupby(["ml_controller","runtime"]).mean().reset_index(),2)
print(table1[table1.columns[4:8]].to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
{} &    mean &   std &           mean &   std \\
\midrule
0 &    6.07 &  0.24 &           4.62 &  0.14 \\
1 &   14.13 &  0.57 &           5.67 &  0.17 \\
2 &    6.63 &  0.22 &           3.95 &  0.10 \\
3 &   14.69 &  0.50 &           5.11 &  0.19 \\
4 &    6.36 &  0.24 &           4.45 &  0.47 \\
5 &   13.81 &  0.40 &           5.61 &  0.49 \\
6 &    6.56 &  0.16 &           4.01 &  0.14 \\
7 &   14.34 &  0.41 &           5.10 &  0.14 \\
\bottomrule
\end{tabular}



In [241]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="Yes"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean(),2)
print(table1[table1.columns[6:8]].to_latex())

\begin{tabular}{llrr}
\toprule
    &     & \multicolumn{2}{l}{ml\_usage} \\
    &     &     mean &    std \\
ml\_controller & speed &          &        \\
\midrule
ARS & 0.5 &    65.65 &   8.98 \\
    & 1.0 &    42.58 &   8.33 \\
    & 1.5 &    26.62 &   5.91 \\
LBC & 0.5 &    83.48 &  13.13 \\
    & 1.0 &    56.77 &  23.67 \\
    & 1.5 &    34.20 &  12.21 \\
SAC & 0.5 &    48.52 &  18.46 \\
    & 1.0 &    15.39 &   7.94 \\
    & 1.5 &     8.12 &   3.99 \\
VBN & 0.5 &    78.63 &  12.33 \\
    & 1.0 &    55.55 &  16.53 \\
    & 1.5 &    40.19 &  11.94 \\
\bottomrule
\end{tabular}



In [242]:
latex=res.reset_index()
#latex[latex.obstacle_presence=="Yes"].groupby(["ml_controller","speed","runtime"]).mean()
table1 = latex[latex.obstacle_presence=="No"]
table1 = round(table1.groupby(["ml_controller","speed"]).mean().reset_index(),2)
print(table1[table1.columns[8:10]].to_latex())

\begin{tabular}{lrr}
\toprule
{} & \multicolumn{2}{l}{ml\_usage} \\
{} &     mean &    std \\
\midrule
0  &    66.79 &   6.33 \\
1  &    48.14 &   6.72 \\
2  &    32.51 &   5.58 \\
3  &    99.59 &   1.12 \\
4  &    95.90 &   8.66 \\
5  &    37.19 &  17.90 \\
6  &    51.70 &  13.35 \\
7  &    11.33 &   9.08 \\
8  &     8.69 &   4.56 \\
9  &    94.20 &   1.00 \\
10 &    82.10 &   9.91 \\
11 &    55.00 &   8.11 \\
\bottomrule
\end{tabular}



In [243]:
withoutobs = all_dfs[all_dfs.obstacle_presence=="No"]
res = round(withoutobs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),2)
res

moet       mean_et       avg_iterations        \
                              mean   std    mean   std           mean   std   
ml_controller speed runtime                                                   
ARS           0.5   10       11.00  2.91    5.99  0.26           4.77  0.05   
                    25       27.24  1.53   14.08  0.87           5.89  0.14   
              1.0   10       14.21  2.39    5.97  0.23           4.99  0.11   
                    25       28.38  2.33   14.07  0.51           5.95  0.17   
              1.5   10        9.13  1.42    6.27  0.24           4.09  0.26   
                    25       27.12  1.72   14.24  0.33           5.17  0.19   
LBC           0.5   10        9.03  0.39    6.60  0.09           3.93  0.02   
                    25       28.25  0.92   14.50  0.11           5.04  0.02   
              1.0   10        9.10  0.49    6.61  0.06           3.93  0.02   
                    25       27.85  0.60   14.57  0.20           5.04  0.03   
              1.5   10        9.67  1.19    6.68  0.51           4.00  0.25   
                    25       27.80  0.76   15.01  1.18           5.27  0.51   
SAC           0.5   10        9.82  2.53    6.37  0.19           4.18  0.35   
                    25       27.38  1.16   13.99  0.31           5.26  0.32   
              1.0   10        9.70  1.75    6.42  0.24           4.44  0.51   
                    25       27.39  1.99   13.76  0.50           5.69  0.61   
              1.5   10        9.38  1.08    6.29  0.28           4.73  0.54   
                    25       27.98  2.11   13.69  0.40           5.89  0.53   
VBN           0.5   10        8.50  0.28    6.54  0.18           3.98  0.01   
                    25       27.17  0.79   14.44  0.70           5.10  0.08   
              1.0   10        8.72  0.38    6.59  0.15           4.02  0.20   
                    25       27.39  0.70   14.21  0.22           5.08  0.16   
              1.5   10        8.93  0.47    6.55  0.17           4.04  0.22   
                    25       27.32  0.94   14.36  0.30           5.13  0.18   

                            ml_usage        sc_usage         
                                mean    std     mean    std  
ml_controller speed runtime                                  
ARS           0.5   10         67.57   6.42    32.43   6.42  
                    25         66.01   6.24    33.99   6.24  
              1.0   10         47.72   7.29    52.28   7.29  
                    25         48.57   6.14    51.43   6.14  
              1.5   10         30.63   7.69    69.37   7.69  
                    25         34.38   3.47    65.62   3.47  
LBC           0.5   10         99.18   2.24     0.82   2.24  
                    25        100.00   0.00     0.00   0.00  
              1.0   10         96.70   2.74     3.30   2.74  
                    25         95.09  14.59     4.91  14.59  
              1.5   10         35.42  18.69    64.58  18.69  
                    25         38.96  17.11    61.04  17.11  
SAC           0.5   10         52.33  13.43    47.67  13.43  
                    25         51.06  13.28    48.94  13.28  
              1.0   10          9.94   8.13    90.06   8.13  
                    25         12.72  10.02    87.28  10.02  
              1.5   10          8.66   4.74    91.34   4.74  
                    25          8.73   4.38    91.27   4.38  
VBN           0.5   10         94.03   1.62     5.97   1.62  
                    25         94.38   0.37     5.62   0.37  
              1.0   10         80.99  13.44    19.01  13.44  
                    25         83.21   6.39    16.79   6.39  
              1.5   10         55.21   5.89    44.79   5.89  
                    25         54.79  10.33    45.21  10.33

In [244]:
withoutobs = all_dfs[all_dfs.obstacle_presence=="No"]
res = round(withoutobs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),2)
print(res[res.columns[:-2]].to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{moet} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} & \multicolumn{2}{l}{ml\_usage} \\
    &     &    &   mean &   std &    mean &   std &           mean &   std &     mean &    std \\
ml\_controller & speed & runtime &        &       &         &       &                &       &          &        \\
\midrule
ARS & 0.5 & 10 &  11.00 &  2.91 &    5.99 &  0.26 &           4.77 &  0.05 &    67.57 &   6.42 \\
    &     & 25 &  27.24 &  1.53 &   14.08 &  0.87 &           5.89 &  0.14 &    66.01 &   6.24 \\
    & 1.0 & 10 &  14.21 &  2.39 &    5.97 &  0.23 &           4.99 &  0.11 &    47.72 &   7.29 \\
    &     & 25 &  28.38 &  2.33 &   14.07 &  0.51 &           5.95 &  0.17 &    48.57 &   6.14 \\
    & 1.5 & 10 &   9.13 &  1.42 &    6.27 &  0.24 &           4.09 &  0.26 &    30.63 &   7.69 \\
    &     & 25 &  27.12 &  1.72 &   14.24 &  0.33 &           5.17 &  0.19 &    34.38 &   3.47 \\
LBC &

# Analysis of Hardware Experiments

In [245]:
hw_file_str = "benchmark_hardware_experiments_{}_{}_{}.csv"
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
hw_dfs = []
for sp in speeds:
    for alg in algorithms:
        for rt in runtimes:
            file_path = hw_file_str.format(sp,alg,rt)
            try:
                df = pd.read_csv(file_path,header=None).rename(columns=cols)
                df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
                df=df.drop(columns=["time_taken_lec","time_taken_safety_controller","total_time","obstacle_presence"])
                df["sc_usage"] = 1 - df["ml_usage"]
                df["ml_usage"]=df["ml_usage"]*100
                df["sc_usage"]=df["sc_usage"]*100
                df['moet'] = df['moet'] * 1000
                df['mean_et'] = df['mean_et'] * 1000
                df["ml_controller"] = alg.replace("_rtreach","")
                df["speed"] = sp
                df['runtime'] = rt
                hw_dfs.append(df)
            except:
                print(file_path)
                continue
hw_dfs = pd.concat(hw_dfs)
hw_dfs["ml_controller"]=hw_dfs["ml_controller"].apply(replace_e2e)
hw_dfs

,time_string,moet,mean_et,avg_iterations,ml_usage,sc_usage,ml_controller,speed,runtime
0,03/04/2021/13:25:54,10.935,5.78091,4.99117,17.917072,82.082928,VBN,0.5,10
1,03/04/2021/13:27:15,11.029,6.30026,4.55825,27.399794,72.600206,VBN,0.5,10
2,03/04/2021/13:28:57,13.587,6.11045,4.89243,26.247901,73.752099,VBN,0.5,10
3,03/04/2021/13:30:38,24.368,5.69501,4.36828,20.250907,79.749093,VBN,0.5,10
4,03/04/2021/13:32:13,12.129,5.33000,4.88028,17.869757,82.130243,VBN,0.5,10
...,...,...,...,...,...,...,...,...,...
0,03/04/2021/13:56:42,24.684,11.01770,5.55328,19.607765,80.392235,ARS,1.0,25
1,03/04/2021/13:58:20,23.122,11.38650,5.33877,15.694671,84.305329,ARS,1.0,25
2,03/04/2021/13:59:56,24.557,11.03970,5.27128,29.077913,70.922087,ARS,1.0,25
3,03/04/2021/14:01:38,25.477,11.30330,5.77496,26.734366,73.265634,ARS,1.0,25


In [246]:
res = round(hw_dfs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),2)
res[res.columns[:-2]]
print(res[res.columns[:-2]].to_latex())

\begin{tabular}{lllrrrrrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{moet} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} & \multicolumn{2}{l}{ml\_usage} \\
    &     &    &   mean &   std &    mean &   std &           mean &   std &     mean &    std \\
ml\_controller & speed & runtime &        &       &         &       &                &       &          &        \\
\midrule
ARS & 0.5 & 10 &  13.81 &  6.39 &    4.88 &  0.26 &           4.90 &  0.71 &    20.22 &   5.30 \\
    &     & 25 &  23.74 &  0.64 &   11.27 &  0.66 &           5.95 &  0.63 &    33.94 &   8.15 \\
    & 1.0 & 10 &  11.21 &  0.87 &    5.09 &  0.42 &           4.64 &  0.46 &    34.24 &  36.91 \\
    &     & 25 &  24.53 &  0.86 &   11.15 &  0.18 &           5.54 &  0.24 &    22.64 &   5.38 \\
LBC & 0.5 & 10 &  13.36 &  2.61 &    6.22 &  0.52 &           4.77 &  0.16 &    21.75 &   6.90 \\
    &     & 25 &  24.82 &  0.99 &   11.34 &  0.64 &           5.37 &  0.19 &    29.15 &   5.57 \\
    &

# Extra Info

In [247]:
hw_file_str = "benchmark_hardware_experiments_{}_{}_{}.csv"
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
hw_dfs = []
for sp in speeds:
    for alg in algorithms:
        for rt in runtimes:
            file_path = hw_file_str.format(sp,alg,rt)
            try:
                df = pd.read_csv(file_path,header=None).rename(columns=cols)
                df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
                df=df.drop(columns=["time_taken_lec","time_taken_safety_controller","obstacle_presence"])
                df["sc_usage"] = 1 - df["ml_usage"]
                df["ml_usage"]=df["ml_usage"]*100
                df["sc_usage"]=df["sc_usage"]*100
                df['moet'] = df['moet'] * 1000
                df['mean_et'] = df['mean_et'] * 1000
                df["ml_controller"] = alg.replace("_rtreach","")
                df["speed"] = sp
                df['runtime'] = rt
                hw_dfs.append(df)
            except:
                print(file_path)
                continue
hw_dfs = pd.concat(hw_dfs)
hw_dfs["ml_controller"]=hw_dfs["ml_controller"].apply(replace_e2e)
hw_dfs['total_time'].describe()

count    83.000000
mean     11.488623
std       3.691032
min       0.208044
25%       8.107125
50%      12.216000
75%      14.487250
max      17.943400
Name: total_time, dtype: float64

In [248]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)
all_dfs = pd.concat(dfs)
all_dfs['total_time'].describe()

count    1822.000000
mean       15.526192
std         5.541865
min         0.749479
25%        10.446225
50%        14.731950
75%        18.613550
max        33.846400
Name: total_time, dtype: float64

## Collisions

In [249]:
collision_files = os.listdir("collisions/")
coll_dfs = []
coll_cols={0:"experiment_number",1:"number_of_cones",2:"object1",
      3:"object2"}
for col_file in collision_files:
    collision_info = col_file.split('_')
    df = pd.read_csv('collisions/'+col_file,header=None).rename(columns=coll_cols).drop_duplicates()
    df['speed'] = collision_info[-3]
    df['runtime']  = collision_info[-2]
    if(len(collision_info)==10):
        df['controller'] =  collision_info[2]+'_'+collision_info[3]
    else:
        df['controller'] = collision_info[2]
    coll_dfs.append(df)
coll_dfs=pd.concat(coll_dfs)
coll_res = coll_dfs.groupby(['speed','controller']).count().reset_index().rename(columns={"experiment_number":"Count"})
coll_res[coll_res.columns[:3]]

,speed,controller,Count
0,0.5,ars,4
1,0.5,e2e,4
2,0.5,e2e_image,10
3,0.5,sac,3
4,1.0,ars,5
5,1.0,e2e,10
6,1.0,e2e_image,10
7,1.0,sac,1
8,1.5,ars,5
9,1.5,e2e,28


In [250]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)
            
all_dfs = pd.concat(dfs)

### Since we conducted 1822 experiments compute how many would occur at 1140

In [251]:
(103/1822)*1440

81.40504939626783

In [98]:
1-(103/1822)

0.9434687156970363

In [99]:
82/1440

0.05694444444444444

In [100]:
1-(82/1440)

0.9430555555555555

In [101]:
1- (103/1822)

0.9434687156970363

### New Hardware Experiments

In [190]:
def summarize_log_file(test_df):
    test_df['mean_et'] = test_df['execution_time'].mean()
    test_df['avg_iterations'] = test_df['num_iterations'].mean()
    test_df['moet'] = test_df['execution_time'].max()
    test_df['ml_usage'] = (test_df['using_safety_controller'].value_counts(normalize=True).sort_values()[0])*100
    test_df['missed_deadline_percent'] = (test_df['missed_deadline'].sum()/test_df.shape[0])*100
    return test_df[['ml_controller','speed','runtime']+list(test_df.columns[9:])].drop_duplicates()

In [191]:
ls hardware_experiments/

ars_0.5_10/  lbc_0.5_10/  old_experiments/  sac_1.0_25/  vbn_1.0_25/
ars_0.5_25/  lbc_0.5_25/  sac_0.5_10/       vbn_0.5_10/
ars_1.0_10/  lbc_1.0_10/  sac_0.5_25/       vbn_0.5_25/
ars_1.0_25/  lbc_1.0_25/  sac_1.0_10/       vbn_1.0_10/


In [252]:
cols={0:"iteration",1:"using_safety_controller",2:"execution_time",
      3:"num_iterations",4:"obstacles"}
algorithms = ["vbn","lbc","sac","ars"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
file_str = "{}_{}_{}"
dfs = []
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = os.path.join("hardware_experiments/",file_str.format(alg,speed,rt))
            log_files = collision_files = os.listdir(file_path)
            for lg in log_files:
                if 'hardware' in lg:
                    continue
                csv_path = os.path.join(file_path,lg)
                try:
                    df = pd.read_csv(csv_path,header=None).rename(columns=cols)
                    if(df.shape[0]<800):
                        continue
                    df['ml_controller'] = alg.upper()
                    df['speed'] = speed
                    df['runtime'] = rt
                    df['execution_time'] = df['execution_time'] *1000
                    df['missed_deadline'] = df['execution_time'].astype(float) > df['runtime'].astype(float)
                    df['missed_deadline'] = df['missed_deadline'].astype(int)
                    new_df = summarize_log_file(df)
                    dfs.append(new_df)
                except:
                    print(csv_path)


In [230]:
hw_exps = pd.concat(dfs)
hw_exps[hw_exps.ml_controller=="ARS"]

,ml_controller,speed,runtime,mean_et,avg_iterations,moet,ml_usage,missed_deadline_percent
0,ARS,0.5,10,4.926047,6.164740,12.875,15.317919,0.674374
0,ARS,0.5,10,4.890078,5.252313,10.318,13.372582,0.420521
0,ARS,0.5,10,5.432109,5.676198,10.809,18.250631,0.084104
0,ARS,0.5,10,4.824064,5.760814,10.534,16.539440,0.932994
0,ARS,0.5,10,4.617146,5.272340,25.082,15.914894,0.170213
0,ARS,0.5,10,5.813062,5.258549,25.793,30.108424,1.084237
0,ARS,0.5,25,9.478095,6.299239,22.478,17.497887,0.000000
0,ARS,0.5,25,9.957513,6.852941,58.602,12.605042,0.084034
0,ARS,0.5,25,9.273231,6.640875,24.693,20.521447,0.000000
0,ARS,0.5,25,10.184133,8.018936,24.494,26.780884,0.000000


In [253]:
hw_exps = pd.concat(dfs)
hw_exps = round(hw_exps.groupby(['ml_controller','speed','runtime']).agg(['mean','std']),2)
hw_exps

mean_et       avg_iterations         moet         \
                               mean   std           mean   std   mean    std   
ml_controller speed runtime                                                    
ARS           0.5   10         5.08  0.45           5.56  0.37  15.90   7.45   
                    25         9.76  0.69           6.77  0.81  29.88  14.10   
              1.0   10         5.45  0.37           4.94  0.49  11.47   1.10   
                    25         8.44  1.63           9.41  1.80  24.71   0.35   
LBC           0.5   10         5.38  0.41           4.55  0.31  23.32  16.14   
                    25         9.71  0.49           6.28  0.89  30.55  13.07   
              1.0   10         5.14  0.34           4.51  0.16  13.72   2.07   
                    25         9.33  0.66           7.47  0.96  28.70  10.46   
SAC           0.5   10         5.12  0.15           5.18  0.71  11.11   0.43   
                    25        11.16  0.92           5.92  0.74  26.70   2.31   
              1.0   10         4.75  0.49           5.40  2.03  11.62   0.59   
                    25         9.71  1.05           7.98  0.87  26.92   5.74   
VBN           0.5   10         5.11  0.49           5.31  0.77  10.89   0.33   
                    25        10.51  0.62           7.41  0.94  24.49   0.47   
              1.0   10         5.60  0.62           5.00  0.15  11.80   2.20   
                    25         7.98   NaN          10.87   NaN  23.56    NaN   

                            ml_usage        missed_deadline_percent        
                                mean    std                    mean   std  
ml_controller speed runtime                                                
ARS           0.5   10         18.25   6.02                    0.56  0.41  
                    25         22.04   6.23                    0.03  0.04  
              1.0   10         20.85   6.34                    1.41  1.23  
                    25         10.30   3.79                    0.01  0.03  
LBC           0.5   10         21.77   7.23                    0.97  0.72  
                    25         24.10   6.84                    0.09  0.09  
              1.0   10         15.71   6.70                    1.01  0.73  
                    25         14.22   5.54                    0.04  0.06  
SAC           0.5   10         16.80   8.49                    2.08  1.09  
                    25         30.41  12.70                    0.09  0.05  
              1.0   10         13.53   3.83                    0.70  0.27  
                    25          9.61   2.79                    0.10  0.12  
VBN           0.5   10         12.46   6.45                    1.06  1.20  
                    25         12.70   2.61                    0.00  0.00  
              1.0   10          9.96   4.68                    2.17  1.66  
                    25          9.58    NaN                    0.00   NaN

In [226]:
analysis = round(pd.concat(dfs),2)
analysis[analysis.ml_usage<100]

,ml_controller,speed,runtime,mean_et,avg_iterations,moet,ml_usage,missed_deadline_percent
0,VBN,0.5,10,4.39,5.97,11.04,8.25,1.10
0,VBN,0.5,10,5.88,4.10,10.84,21.19,0.47
0,VBN,0.5,10,5.33,4.91,11.00,16.26,3.45
0,VBN,0.5,10,5.11,5.64,10.50,2.94,0.45
0,VBN,0.5,10,5.01,6.17,11.38,15.07,0.44
...,...,...,...,...,...,...,...,...
0,ARS,1.0,25,7.11,10.85,24.54,9.24,0.00
0,ARS,1.0,25,8.32,10.19,24.56,7.36,0.00
0,ARS,1.0,25,10.29,7.40,24.86,10.46,0.00
0,ARS,1.0,25,10.08,8.47,24.90,8.57,0.00


In [227]:
test_df.agg(
    {
    "execution_time": ['mean','std'],
    "num_iterations": ['mean','std']}
).T

,mean,std
execution_time,8.720041,5.988675
num_iterations,7.722911,6.209516
